# Likelihood Optimization of gas Kinematics in IFUs (LOKI)
## Fitting example: MIRI

Michael Reefe

This example notebook provides a quick tutorial on how to run LOKI using JWST MIRI/MRS formatted data.

First things first, we need to import the LOKI code. Remember we need to activate our project first (refer to the installation section of the README). We can do so simply by:

In [1]:
using Pkg
Pkg.activate(dirname(@__DIR__))
Pkg.instantiate()
Pkg.precompile()

using Loki

  Activating project at `~/Dropbox/Astrophysics/Phoenix_Cluster/Loki`
Precompiling packages...
  12874.1 ms  ✓ Loki
  1 dependency successfully precompiled in 15 seconds. 308 already precompiled.



Some aspects of the code may utilize multiprocessing. To take advantage of this, we first must import the `Distributed` package and add parallel CPU processes. Then, our following imports must be encased in an `@everywhere` block to ensure they are loaded onto each CPU process individually. For example:

```julia
@everywhere begin
    using Pkg
    Pkg.activate(dirname(@__DIR__))
    Pkg.instantiate()
    Pkg.precompile()
    using Loki
end
```

### NOTE 1: 

Alternatively, we could have started julia with the command-line argument `--project=/path/to/Loki`. This is my preferred way of starting the code when in a project setting, but this is difficult to demonstrate using a Jupyter notebook. If you choose to start julia this way, the above block is unnecessary since the project is already activated and precompiled. You would just need to add a single line like `@everywhere using Loki`. Additionally, you would have to modify the `addprocs` call with the argument `exeflags="--project=/path/to/Loki"`, which tells the Distributed module that the worker processes should also be started using the Loki project.  

So, all together, in your file (let's call it `example.jl`) you would have:
```julia
using Distributed
procs = addprocs(Sys.CPU_THREADS, exeflags="--project=/path/to/Loki")
@everywhere using Loki
```
And from the terminal you would start the code using:
`julia --project=/path/to/Loki example.jl`

### NOTE 2:

If you plan on running Loki in a High-Performance Computing (HPC) environment, you can also take advantage of julia's built-in distributed package in a very similar way. In fact, if you plan on running on only a single node, then no changes should be necessary at all from the above example. However, if you wish to spread out over multiple nodes, some changes will be required, since by default the above will only add additional processes onto one node. If you cluster uses Slurm, then you can use the `SlurmClusterManager` package like so:

```julia
using Distributed
using SlurmClusterManager
procs = addprocs(SlurmManager(), exeflags="--project=/path/to/Loki")
@everywhere using Loki
```

Then, you would start julia from within an sbatch script specifying how many nodes, tasks, and CPUs you need. I strongly recommend using the `"--project"` flag approach described above when on a cluster, as I have run into strange issues with the other approach causing julia to start thinking pid files are stale and removing them.

Now we want to load in our data. For this example, we'll be using the channel 1 data for NGC 7469, which is located in the same folder as this notebook. Unfortunately the JWST reduced data does not include a redshift, so we must provide the redshift ourselves.  We can use the `from_fits` function to load in the JWST-formatted FITS files, along with the redshift.

In [2]:
# The redshift of the target object: NGC 7469
z = 0.016317
# The semicolon at the end suppresses printing the output Observation object, which is long and not very enlightening
# obs = from_fits(["Level3_ch1-long_s3d.fits", "Level3_ch1-medium_s3d.fits", "Level3_ch1-short_s3d.fits"], z);
obs = from_fits(["Level3_ch1-medium_s3d.fits"], z);

[ Info: Initializing DataCube struct from Level3_ch1-medium_s3d.fits


Next, we create some variables that we will use later. We will be fitting channel 1 data, and we can take the `name` property from the Observation object we just loaded in to get the name of the target. Here, `run_name` is just a unique identifier that we will use for this run.

In [3]:
# channel = 1
channel = :B1
nm = replace(obs.name, " " => "_") 
run_name = "$(nm)_ch$(channel)_nuc_aperture"

"NGC_7469_chB1_nuc_aperture"

Before fitting, we want to do some pre-processing on the data. We want to convert the data to the rest-frame, mast out / interpolate any bad pixels, and replace the JWST pipeline-generated errors with some more realistic ones.  All of this is achieved in the next block of code. This is also where we could combine data from multiple channels into a single cube, if desired, using the `combine_channels!` function, but in this quick example we only have one sub-channel.

In [7]:
if isfile("$nm.channel$channel.rest_frame.fits")
    # If we've already performed this step in a previous run, just load in the pre-processed data
    obs = from_fits(["$nm.channel$channel.rest_frame.fits"], obs.z);
    
else
    # Convert to rest-frame wavelength vector, and mask out bad spaxels
    correct!(obs)
    
    # Reproject the sub-channels onto the same WCS grid and combine them into one full channel
    # - The [:A1, :B1, :C1] vector gives the names of each channel to concatenate. By default, JWST subchannels are
    #   given labels of "A" for short, "B" for medium, and "C" for long, followed by the channel number.  
    # - The "out_id" argument will determine the label given to the combined channel data. 
    # combine_channels!(obs, [:A1,:B1,:C1], out_id=channel)

    # the input data cubes are already in the sky frame, so we dont need to use the rotate_to_sky_axes! function
    
    # We interpolate any rogue NaNs using a linear interpolation, since the MPFIT minimizer does not handle NaNs well.
    interpolate_nans!(obs.channels[channel])

    # Finally, we calculate the statistical errors (i.e. the standard deviation of the residuals with a cubic spline fit)
    # and replace the errors in the cube with these, since the provided errors are typically underestimated.
    # You can skip this step if you wish to use the default errors.
    calculate_statistical_errors!(obs.channels[channel])
    
    # Save the pre-processed data as a FITS file so it can be quickly reloaded later
    save_fits(".", obs, [channel]);
end

[ Info: Initializing DataCube struct from NGC_7469.channelB1.rest_frame.fits


Observation(Dict{Any, DataCube}(:B1 => DataCube{Vector{Unitful.Quantity{Float64, 𝐋, Unitful.FreeUnits{(μm,), 𝐋, nothing}}}, Array{Unitful.Quantity{Float64, 𝐌 𝐓⁻², Unitful.FreeUnits{(erg, Hz⁻¹, cm⁻², s⁻¹, sr⁻¹), 𝐌 𝐓⁻², nothing}}, 3}}(Unitful.Quantity{Float64, 𝐋, Unitful.FreeUnits{(μm,), 𝐋, nothing}}[5.569521957619527 μm, 5.570309113575583 μm, 5.57109638078257 μm, 5.57188375925621 μm, 5.572671249012229 μm, 5.573458850066356 μm, 5.57424656243432 μm, 5.575034386131854 μm, 5.575822321174693 μm, 5.57661036757857 μm  …  6.514537321833719 μm, 6.515458039427231 μm, 6.516378887148309 μm, 6.517299865015345 μm, 6.518220973046733 μm, 6.519142211260869 μm, 6.52006357967615 μm, 6.520985078310981 μm, 6.521906707183765 μm, 6.52282846631291 μm], Unitful.Quantity{Float64, 𝐌 𝐓⁻², Unitful.FreeUnits{(erg, Hz⁻¹, cm⁻², s⁻¹, sr⁻¹), 𝐌 𝐓⁻², nothing}}[NaN erg Hz⁻¹ cm⁻² s⁻¹ sr⁻¹ NaN erg Hz⁻¹ cm⁻² s⁻¹ sr⁻¹ … NaN erg Hz⁻¹ cm⁻² s⁻¹ sr⁻¹ NaN erg Hz⁻¹ cm⁻² s⁻¹ sr⁻¹; NaN erg Hz⁻¹ cm⁻² s⁻¹ sr⁻¹ NaN erg Hz⁻¹ cm⁻² s⁻¹ sr⁻¹

We next create an aperture to define the region of interest that we would like to fit. We can do this with the `make_aperture` function. We can customize the aperture's shape, centroid, radius, etc.

In [8]:
# - The first argument is the data cube
# - The second argument is the aperture shape, which may be one of: (Circular, Rectangular, Elliptical)
# - Next are the right ascension in sexagesimal hours and the declination in sexagesimal degrees
# - The next arguments depend on the aperture shape:
#    - For circles, it is the radius in arcseconds
#    - For rectangles, it is the width in arcseconds, height in arcseconds, and rotation angle in degrees
#    - For ellipses, it is the semimajor axis in arcseconds, semiminor axis in arcseconds, and rotation angle in degrees
# - The auto_centroid argument, if true, will adjust the aperture centroid to the closest peak in brightness
# - The scale_psf argument, if true, will create a series of apertures with increasing radii that scale at the same rate as the PSF
ap = make_aperture(obs.channels[channel], :Circular, "23:03:15.610", "+8:52:26.10", 0.5, auto_centroid=true)

[ Info: Creating a circular aperture at 23:03:15.610, +8:52:26.10
[ Info: Aperture centroid adjusted to 0h56m44.384367413044856s, 8d52m26.122293063559994s


8×9 Photometry.Aperture.CircularAperture{Float64} with indices 14:21×16:24:
 0.0          0.0       0.240967   …  0.501674  0.0272194  0.0
 0.0          0.280688  0.983525      1.0       0.706297   0.00430573
 0.000634748  0.847435  1.0           1.0       1.0        0.294459
 0.0917088    1.0       1.0           1.0       1.0        0.538098
 0.0663343    0.997272  1.0           1.0       1.0        0.509995
 0.0          0.756991  1.0        …  1.0       0.997322   0.206058
 0.0          0.161489  0.915917      0.999973  0.523823   0.0
 0.0          0.0       0.0962037     0.28933   0.0        0.0

Finally, we create the `CubeFitter` object and call the `fit_cube!` function to fit the data. To ensure we fit the data within the aperture, we must provide the `ap` argument.

If you instead wish to fit each spaxel individually, you may omit the `ap` argument. Be warned that this will take substantially longer. If you wish to fit each spaxel individually, it is recommended to enable the "parallel" option to allow multiple spaxels to be fit simultaneously. Doing this will also allow the code to produce 2D parameter maps of each fit parameter.

In [9]:
# To see a full list of keyword arguments, please refer to the docstring, which can be accessed by typing `?CubeFitter` in the command
# line after importing Loki.
cube_fitter = CubeFitter(obs.channels[channel], obs.z, run_name; parallel=false, plot_spaxels=:both, 
    plot_maps=true, save_fits=true, use_pah_templates=false, fit_sil_emission=true, save_full_model=false,
    fit_stellar_continuum=false)

# Call the fit_cube! function on the cube_fitter object, using the aperture we defined.
fit_cube!(cube_fitter, ap)

[ Info: Preparing output directories
┌ Info: 
│ 
│ #############################################################################
│ ######## BEGINNING FULL CUBE FITTING ROUTINE FOR NGC_7469_chB1_nuc_aperture ########
│ #############################################################################
│ 
│ ------------------------
│ Worker Processes:     1
│ Threads per process:  1
└ ------------------------
[ Info: ===> Preparing output data structures... <===
[ Info: Performing aperture photometry to get an integrated spectrum...
[ Info: ===> Beginning integrated spectrum fitting... <===
┌ Warning: attempting to remove probably stale pidfile
│   path = "/Users/mreefe/.jlassetregistry.lock"
└ @ Pidfile ~/.julia/packages/Pidfile/DDu3M/src/Pidfile.jl:260
[ Info: ===> Generating parameter maps and model cubes... <===
[ Info: ===> Writing FITS outputs... <===
┌ Info: 
│ 
│ #############################################################################
│ ################################### Done!! #

(CubeFitter{Float64, Int64, Unitful.Quantity{Float64, 𝐌 𝐓⁻², Unitful.FreeUnits{(erg, Hz⁻¹, cm⁻², s⁻¹, sr⁻¹), 𝐌 𝐓⁻², nothing}}, Unitful.Quantity{Float64, 𝐋 𝐓⁻¹, Unitful.FreeUnits{(km, s⁻¹), 𝐋 𝐓⁻¹, nothing}}, Unitful.Quantity{Float64, 𝐋, Unitful.FreeUnits{(μm,), 𝐋, nothing}}}(DataCube{Vector{Unitful.Quantity{Float64, 𝐋, Unitful.FreeUnits{(μm,), 𝐋, nothing}}}, Array{Unitful.Quantity{Float64, 𝐌 𝐓⁻², Unitful.FreeUnits{(erg, Hz⁻¹, cm⁻², s⁻¹, sr⁻¹), 𝐌 𝐓⁻², nothing}}, 3}}(Unitful.Quantity{Float64, 𝐋, Unitful.FreeUnits{(μm,), 𝐋, nothing}}[5.569521957619527 μm, 5.570309113575583 μm, 5.57109638078257 μm, 5.57188375925621 μm, 5.572671249012229 μm, 5.573458850066356 μm, 5.57424656243432 μm, 5.575034386131854 μm, 5.575822321174693 μm, 5.57661036757857 μm  …  6.514537321833719 μm, 6.515458039427231 μm, 6.516378887148309 μm, 6.517299865015345 μm, 6.518220973046733 μm, 6.519142211260869 μm, 6.52006357967615 μm, 6.520985078310981 μm, 6.521906707183765 μm, 6.52282846631291 μm], Unitful.Quantity{Float64, 

And the results can be found in the "output_[run_name]" directory!
Here is our fit of channel 1B of the nuclear spectrum of NGC 7469\*:

![results_1D](./NGC_7469_spaxel_1_1.nucleus.jpg)

The orange line shows the final model.  The decomposed components of the model consist of:
- Thermal dust continuum, in gray
- Stellar continuum, in pink
- Hot silicate dust emission, in green
- PAHs, in blue
- Emission lines, in purple
- Extinction, in dotted gray (read from the right axis)

The thicker gray line that lies close to the final orange model is showing the combined continuum components from the dust and stars (but excluding the PAHs).

You will also notice that emission lines get nice labels at the top of the plot.

\*Side note: it is generally not a good idea to fit such a small wavelength range with just this one sub-channel, this was done just as a quick example to get you started. The continuum components can be degenerate with the very flat extinction profile if you happen to land on a relatively featureless part of the continuum (such as the section shown here). This is less of a problem at shorter wavelengths, due to the complicated shape of the stellar continuum, and around 10-18 microns where there are large silicate absorption features. If you absolutely must fit a part of the spectrum like the one shown here, just take the return values for the continuum and extinction with a grain of salt.